In [2]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
import os
from datetime import datetime

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] ='''
--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,org.apache.spark:spark-streaming-kafka-0-10-assembly_2.12:3.4.1,org.apache.spark:spark-avro_2.12:3.4.1,io.delta:delta-core_2.12:2.4.0 --conf spark.sql.session.timeZone=GMT pyspark-shell'''

In [4]:
spark_session = SparkSession.builder \
            .appName("read_hdfs") \
            .master("local") \
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()

23/11/25 23:03:24 WARN Utils: Your hostname, pm-GL65-Leopard-10SCXK resolves to a loopback address: 127.0.1.1; using 192.168.31.92 instead (on interface wlo1)
23/11/25 23:03:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pm/Downloads/spark-3.4.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pm/.ivy2/cache
The jars for the packages stored in: /home/pm/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-10-assembly_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d920e67f-b326-451a-8aac-15c55d8be9f9;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in local-m2-c

In [5]:
engine = create_engine("postgresql+psycopg2://bd:bd@localhost/stock?client_encoding=utf8")

In [6]:
year, month, day = datetime.now().year, datetime.now().month, datetime.now().day

In [7]:
spark_session.conf.set("spark.sql.streaming.schemaInference", True)


In [41]:
def get_connection():
   try:
       return psycopg2.connect(
           database="historical",
           user="bd",
           password="bd",
           host="localhost",
           port=5432,
       )
   except:
       return False

conn = get_connection()

In [42]:
curr = conn.cursor()
curr.execute("SELECT * FROM HNX30;")
data = curr.fetchall()
data

UndefinedTable: relation "hnx30" does not exist
LINE 1: SELECT * FROM HNX30;
                      ^


In [24]:
df = spark_session.read.load("hdfs://localhost:9000/delta/stock1/symbol=BNA/year=2023/month=09/day=27")

In [25]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [26]:
df.show()

+-------------+-------------------+---------+----------+-----------+--------+---------+--------+---------+--------+---------+----------+----------+---------+---------------+----------------------+--------+---------+--------+---------+--------+---------+----------+------+---------+---------+-----------+------------+-----------+----+
|exchange_name|         trade_time|ref_price|ceil_price|floor_price|b_qtty_3|b_price_3|b_qtty_2|b_price_2|b_qtty_1|b_price_1|pct_change|last_price|last_qtty|traded_quantity|total_value_traded_qtt|s_qtty_3|s_price_3|s_qtty_2|s_price_2|s_qtty_1|s_price_1|open_price|mid_px|max_price|min_price|foreign_buy|foreign_sell|close_price|hour|
+-------------+-------------------+---------+----------+-----------+--------+---------+--------+---------+--------+---------+----------+----------+---------+---------------+----------------------+--------+---------+--------+---------+--------+---------+----------+------+---------+---------+-----------+------------+-----------+----

In [51]:
engine = create_engine('postgresql://bd:bd@localhost:5432/historical')
df.to_sql('vn30', engine, if_exists='replace')

720